In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *

In [3]:
path = Path('data/kaggle/cactus/')

In [4]:
path.ls()

[WindowsPath('data/kaggle/cactus/models'),
 WindowsPath('data/kaggle/cactus/sample_submission.csv'),
 WindowsPath('data/kaggle/cactus/test'),
 WindowsPath('data/kaggle/cactus/test.zip'),
 WindowsPath('data/kaggle/cactus/train'),
 WindowsPath('data/kaggle/cactus/train.csv'),
 WindowsPath('data/kaggle/cactus/train.zip')]

In [5]:
test_df = pd.read_csv(path/'SAMPLE_SUBMISSION.csv')
train_df = pd.read_csv(path/'train.csv')

In [6]:
test_img = ImageList.from_df(test_df, path=path, folder='test')

In [7]:
tfms = get_transforms(flip_vert=True, max_rotate=10.0, max_zoom=1.1, max_lighting=0.2, max_warp=0.2, p_affine=0.75, p_lighting=0.75)
train_img = ImageList.from_df(train_df, path =path, folder='train')

In [8]:
train_img = train_img.split_by_rand_pct(0.01).label_from_df().add_test(test_img).transform(tfms, size=128).databunch(path=path, bs=48).normalize(imagenet_stats)

In [9]:
learn = cnn_learner(train_img, models.densenet161, metrics=accuracy)

In [10]:
learn.fit_one_cycle(8, slice(3e-02))

epoch,train_loss,valid_loss,accuracy,time
0,0.046552,0.026474,0.994286,04:50
1,0.061206,0.009260,0.994286,04:35
2,0.030106,0.025970,0.988571,04:49
3,0.036222,0.000017,1.000000,04:49
4,0.016741,0.000171,1.000000,04:43
5,0.008566,0.000196,1.000000,05:02
6,0.005280,0.000078,1.000000,04:43
7,0.002982,0.000076,1.000000,04:39


In [12]:
pres,_ = learn.get_preds(ds_type=DatasetType.Test)

In [13]:
test_df.has_cactus = pres.numpy()[:,0]
test_df.to_csv(path/'sub.csv')

In [ ]:
gc.collect()

In [ ]:
accuracy_score(df.class, df.pr)